### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses Natural Language processing combined with REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory. Selenium needs a webdriver installed, find details at https://sites.google.com/chromium.org/driver/downloads

### NOTE selenium cache files are quite large and need to be cleaned up periodically

In [ ]:
pip install "numpy<2.0" 

In [ ]:
pip install spacy
#pip install -U pip setuptools wheel
#pip install -U spacy
#pip install --upgrade spacy

### Install the model at the command lines
small model
python -m spacy download en_core_web_sm

medium model
python -m spacy download en_core_web_md

large model
python -m spacy download en_core_web_lg

### create training set https://spacy.io/usage/training

This annotator works better https://agateteam.org/spacynerannotate/ combined with the DocBin code to create the .spacy file. 

https://spacy.io/usage/training#quickstart

python -m spacy init fill-config ./base_config.cfg ./config.cfg

This NER annotator worked better than the one above. Used the DocBin technique to convert to .spacy file. Spacy training command with config.cfg then worked.

Config.cfg needs to be created for the spacy command line training. 

Follow the instructions in the quickstart for the base_config.cfg file. Selected OS and clicked NER

https://spacy.io/usage/training#quickstart

Run the following at command line to create the config.cfg. 

python -m spacy init fill-config ./base_config.cfg ./config.cfg

Once the config file was done and spacy file was created run the following at command line to train the model

To debug the data:
python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy

To debug the config file file:
python -m spacy train config.cfg --output ./output

or

python -m spacy train config.cfg 

train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

creates two directories model-best and model-last. I used model-best 

nlp1 = spacy.load(r"./output/model-best")


Notes
python -m spacy init fill-config ./base_config.cfg ./config.cfg

python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

https://spacy.io/usage/training#basics

### Train spacy if not already done. Using output https://arunmozhi.in/ner-annotator/


In [ ]:
from spacy.tokens import DocBin
import pandas as pd
import json
import os

### train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy 

### Main webscraper program start


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import IPython
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName
import spacy
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")
from spacy.matcher import PhraseMatcher
from spacy import displacy

In [ ]:
#test
text2 = ("The type strain is 2205BS29-5T (=LMG 33062T =KACC 23240T), which is isolated from a marine sponge, P. elegans, "
"collected from Beomseom in Jeju-Island, Republic of Korea."

"The DNA G+C content of strain 2205BS29-5T is 67.8%. The GenBank accession numbers for the 16S rRNA gene and" 
"whole-genome sequences of strain 2205BS29-5T are OQ569368 and JAVAMQ000000000, respectively.")

In [ ]:
#test but need this later
import IPython
nlp1 = spacy.load(r"./output/model-best") #load the best model
doc = nlp1(text2)
from IPython.display import display
from spacy import displacy
from IPython.core.display import HTML, display
displacy.render(doc, style="ent")

### Find URLS from saved email in html - save as from outlook in htm format. URLs are extracted and saved as input for selenium

In [2]:
input = (r'IJSEMemail54.htm')
output = (r'NameCheckweek54.xlsx')
alldescriptions = (r'all_descriptions54')

In [ ]:
#Only need this if you suspect a problem with beautiful soup not imporing the URL list correctly. Otherwise skip this step
from bs4.diagnose import diagnose
with open (input, encoding = 'unicode_escape') as f:
    data = f.read()
diagnose(data)

#### alternative Beautifiul soup code for extracting URLS with autodetect encoding

In [3]:
#alternative Beautifiul soup with autodetect encoding
from charset_normalizer import from_path

# Auto-detect file encoding
result = from_path(input).best()
html = str(result)

# Parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
text = soup.get_text()

# Extract all http/https links
urls = re.findall(r"https?://\S+", text)
urls = [url.rstrip('.,);') for url in urls]

# Filter links as needed
filtered_urls = [
    url for url in urls
    if all(exclude not in url for exclude in ["TandC", "myaccount", "join-the-society"])
    # Remove "doi.org" from filter if you want article links
]

if not filtered_urls:
    raise ValueError("No usable URLs found!")

print(f"Found {len(filtered_urls)} URLs:")
print("\n".join(filtered_urls))


Found 13 URLs:
http://dx.doi.org/10.1099/ijsem.0.006836
http://dx.doi.org/10.1099/ijsem.0.006850
http://dx.doi.org/10.1099/ijsem.0.006839
http://dx.doi.org/10.1099/ijsem.0.006837
http://dx.doi.org/10.1099/ijsem.0.006840
http://dx.doi.org/10.1099/ijsem.0.006753
http://dx.doi.org/10.1099/ijsem.0.006834
http://dx.doi.org/10.1099/ijsem.0.006814
http://dx.doi.org/10.1099/ijsem.0.006819
http://dx.doi.org/10.1099/ijsem.0.006813
http://dx.doi.org/10.1099/ijsem.0.006833
http://dx.doi.org/10.1099/ijsem.0.006828
http://dx.doi.org/10.1099/ijsem.0.006838


In [4]:
def find_strains(description):
    """Find strains with spacy"""
    #print('Entering strain subroutine')
    doc = nlp(description)
    nlp1 = spacy.load(r"./output/model-best") #load the best model
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrases = ['strain', 'Strain', 'strains', 'Strains']
    patterns = [nlp(description) for description in phrases]
    phrase_matcher.add('STRAIN', None, *patterns)
               
    for sent in doc.sents:    
        for match_id, start, end in phrase_matcher(nlp(sent.text)):
            if nlp.vocab.strings[match_id] in ["STRAIN"]:
                print(sent.text)
                doc = nlp1(sent.text)
                displacy.render(doc, style="ent")
                for entity in doc.ents:
                    if entity.text not in strains:
                        entity.text.encode('ascii', 'ignore').decode('unicode_escape')
                        strains.append(entity.text)
    return strains

def remove_non_ascii(text):
    """Remove non-ASCII characters"""
    return ''.join(char for char in text if ord(char) < 128)

### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [5]:
#from selenium.webdriver.common.by import By
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'filtered_url'])
pd.set_option('display.max_columns', None)
combined_description = []
for filtered_url in filtered_urls:
    counter = 1
    strains = []
    accessions = []
    orgname = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    description1 = []
    description2 = []
    snumber = []
    #scrape_with_selenium(filtered_url, counter)
    #scrape_with_beautifulsoup(filtered_url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(filtered_url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from each species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description:
            strains = []
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                cleaned_text = remove_non_ascii(description)
                combined_description.append(cleaned_text)
                #print(cleaned_text)
                #print(description)
                        
                #find the organism names 
                orgname = []
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                if description is not None:
                    orgname = [m.group() for m in regex.finditer(description)]
                    print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    #print('accessions', accessions)
    
                #find the strains
                if description is not None:
                    find_strains(cleaned_text)
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, filtered_url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK1')

   
    for element in driver.find_elements(By.CLASS_NAME, "tl-lowest-section"): #finds section headers
        description1 = element.text
        outer_html = element.get_attribute("outerHTML")
        if "Description of" in description1:          
            #print(outer_html)
            spans = soup.find_all('span', attrs = {'class' : 'tl-lowest-section'})
            for span in spans:
                if "Description of" in span.text:   
                    #print (span.text)
                    outer_div_id = span.find_parent('div').get('id')
                    print(f"Outer div ID: {outer_div_id}, Text: {span.text}")
                    for element in driver.find_elements(By.ID, outer_div_id):
                        description = element.text
                        cleaned_text = remove_non_ascii(description)
                        combined_description.append(cleaned_text)
                        #print(cleaned_text)
                        #print(description)
                        
                    #find the organism names   
                    orgname = []
                    match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                    if description is not None:
                        orgname = [m.group() for m in regex.finditer(description)]
                        #print('orgname', orgname)

                    #find the accessions
                    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                    if description is not None:
                        accessions = [m.group() for m in regex.finditer(description)]
                        #print('accessions', accessions)
    
                    #find the strains
                    strains = []
                    if description is not None:
                        find_strains(cleaned_text)
                        #print('strain names', strains)
    
                    #load data into pandas dataframe
                    row_data = [orgname, accessions, strains, authority, doi, filtered_url]
                    length = len(pub_df)
                    pub_df.loc[length] = row_data
                    print('BREAK2')



        
#Close the browser window
    driver.quit()    

Rhodotorula silvicola sp. nov., a new yeast species from plant-associated substrates and mushroom
orgname ['F.Y. Bai ', 'R. silvicola ']
Neither pseudohyphae nor true hyphae are formed on CMA and PDA after 10days at 20C. Additionally, sexual structure was not observed in the single and pairwise mixed strain cultures of strain CGMCC 2.7770 and strain UCDFST 81-84 on different media, including CMA, PDA, YM, YCBAS and V8, after incubation at 20C for 2months.



Neither pseudohyphae nor true hyphae are formed on CMA and PDA after 10days at 20C. Additionally, sexual structure was not observed in the single and pairwise mixed strain cultures of strain CGMCC 2.7770 and strain UCDFST 81-84 on different media, including CMA, PDA, YM, YCBAS and V8, after incubation at 20C for 2months.



Neither pseudohyphae nor true hyphae are formed on CMA and PDA after 10days at 20C. Additionally, sexual structure was not observed in the single and pairwise mixed strain cultures of strain CGMCC 2.7770 and strain UCDFST 81-84 on different media, including CMA, PDA, YM, YCBAS and V8, after incubation at 20C for 2months.



The morphology of R. silvicola sp. nov. strain CGMCC 2.7770T observed on YPD agar after 1week at 20C includes colony morphology (a) and vegetative cells (b).


Growth on YPD agar is positive at 30C but negative at 37C.
The holotype, CGMCC 2.7770T (original strain number 1017-17-3), was isolated from rotten leaves from Zunyi City, Guizhou Province, PR China, in September 2023, and has been deposited in a metabolically inactive state in the China General Microbiological Culture Collection Center, Beijing, PR China.


...

The major polar lipids of strain NT6NT were PE, DPG and PG.


The G+C content of the genomic DNA of strain NT6NT is 52.1mol%.


The DDBJ/GenBank/EMBL accession numbers for the 16S rRNA gene sequence and whole-genome sequence of strain NT6NT are LC820896 and AP026866.1, respectively.



The type strain is NT6NT (=


strain names ['NT6NT']
orgname ['Oceaniferula spumae ']
strain names ['NT6NT']
BREAK1
Corrigendum: Faecalibacterium langellae sp. nov. isolated from human faeces


In [6]:
#optional write description to a file
#print(combined_description)
file = open(alldescriptions, "w")
file.writelines(combined_description)
file.close()

In [7]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [', '.join(map(str, l)) for l in pub_df['Strains']]
#pub_df['Strains'] = pub_df['Strains'].astype(str) 
pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#try drop duplicate accessions here
pub_df.explode(['PublishedName']).reset_index(drop=True)

,PublishedName,Accessions,Strains,Authority,DOI,filtered_url
0,F.Y. Bai,"[PQ432548, PQ432547]","CGMCC 2.7770, UCDFST 81-84, PDA, YM, CGMCC 2.7770T, 1017-17-3",Wei et al. 2025,https://doi.org/10.1099/ijsem.0.006836,http://dx.doi.org/10.1099/ijsem.0.006836
1,R. silvicola,"[PQ432548, PQ432547]","CGMCC 2.7770, UCDFST 81-84, PDA, YM, CGMCC 2.7770T, 1017-17-3",Wei et al. 2025,https://doi.org/10.1099/ijsem.0.006836,http://dx.doi.org/10.1099/ijsem.0.006836
2,F.Y. Bai,[],"CGMCC 2.7770, UCDFST 81-84, PDA, YM, CGMCC 2.7770T, 1017-17-3",Wei et al. 2025,https://doi.org/10.1099/ijsem.0.006836,http://dx.doi.org/10.1099/ijsem.0.006836
3,Farysia photiniae,"[OP566885, OP566881, PV170748, PV170750, PV170746, PV172328, PV172327, PV170749, PV170751, PV170747]","PYCC 9932, NYNU 22992",Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006850,http://dx.doi.org/10.1099/ijsem.0.006850
4,NaN,[],,Wu et al. 2025,https://doi.org/10.1099/ijsem.0.006839,http://dx.doi.org/10.1099/ijsem.0.006839
5,Methanosuratincola petrocarbonis,[OR243905],"LWZ-6T, CCAM 1872T, JCM 39528T",Wu et al. 2025,https://doi.org/10.1099/ijsem.0.006839,http://dx.doi.org/10.1099/ijsem.0.006839
6,Brachymonas wangyanguii,"[PP212021, PQ632502, CP173677, JBBEWD000000000]","G13T, GDMCC 1.4691T, JCM 37746T",Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006837,http://dx.doi.org/10.1099/ijsem.0.006837
7,Fodinibius alkaliphilus,"[PP669793, JBCFYD000000000]","N2T, KCTC 102228T, MCCC 1K08942T",He et al. 2025,https://doi.org/10.1099/ijsem.0.006840,http://dx.doi.org/10.1099/ijsem.0.006840
8,Fodinibius salipaludis,"[MF782427, NSKE00000000]","WN023T, KCTC 52855T, ACCC 19978T",He et al. 2025,https://doi.org/10.1099/ijsem.0.006840,http://dx.doi.org/10.1099/ijsem.0.006840
9,Chlamydia vaughanii,[OZ026853],"ATCC CRL-1696, EPC, ATCC CRL-2872",Marquis et al. 2025,https://doi.org/10.6084/m9.figshare.28424588.v,http://dx.doi.org/10.1099/ijsem.0.006753


In [ ]:
#pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
#pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df

In [8]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [9]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [10]:
#df_unique['accession'] = df_unique['accession'].astype('str') 
df_unique.loc[:, 'accession'] = df_unique['accession'].astype('str') 

In [11]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [12]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


Error: Malformatted ID "nan"


0

In [13]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna(subset=['NCBIname'])
srcchk_df 

,accession,NCBIname,taxid,strain
0,PQ432548,Rhodotorula sp.,1853554.0,1017-17-3
1,PQ432547,Rhodotorula sp.,1853554.0,1017-17-3
3,OP566885,Farysia sp.,2012362.0,NYNU 22992
4,OP566881,Farysia sp.,2012362.0,NYNU 22992
5,PV170748,Farysizyma sp.,2650697.0,NYNU 22992
6,PV170750,Farysizyma sp.,2650697.0,NYNU 22992
7,PV170746,Farysizyma sp.,2650697.0,NYNU 22992
8,PV172328,Farysia sp.,2012362.0,NYNU 229246
9,PV172327,Farysia sp.,2012362.0,NYNU 229246
10,PV170749,Farysizyma sp.,2650697.0,NYNU 229246


### Combine dataframes into one

In [15]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'filtered_url' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,filtered_url
0,Oceaniferula spumae,Oceaniferula spumae,NT6NT,AP026866,NT6N,Huang et al. 2025,2979115.0,https://doi.org/10.1099/ijsem.0.006828,http://dx.doi.org/10.1099/ijsem.0.006828
1,Streptococcus tangpeifui,Streptococcus sp. ZJ 1593,"ZJ1593T, GDMCC 1.3115T, JCM 35327T, ZJ373",CP172427,ZJ 1593,Pu et al. 2025,3366265.0,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
2,Brachymonas wangyanguii,Brachymonas sp. G13,"G13T, GDMCC 1.4691T, JCM 37746T",CP173677,G13,Gao et al. 2025,3130163.0,https://doi.org/10.1099/ijsem.0.006837,http://dx.doi.org/10.1099/ijsem.0.006837
3,Streptococcus tangpeifui,Streptococcus tangpeifui,"ZJ1593T, GDMCC 1.3115T, JCM 35327T, ZJ373",JAAKDU000000000,ZJ373,Pu et al. 2025,2709400.0,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
4,Brachymonas wangyanguii,Brachymonas wangyanguii,"G13T, GDMCC 1.4691T, JCM 37746T",JBBEWD000000000,G34,Gao et al. 2025,3130163.0,https://doi.org/10.1099/ijsem.0.006837,http://dx.doi.org/10.1099/ijsem.0.006837
5,Fodinibius alkaliphilus,Fodinibius alkaliphilus,"N2T, KCTC 102228T, MCCC 1K08942T",JBCFYD000000000,N2,He et al. 2025,3140241.0,https://doi.org/10.1099/ijsem.0.006840,http://dx.doi.org/10.1099/ijsem.0.006840
6,Streptococcus jiangjianxini,Streptococcus jiangjianxini,"ZJ151T, GDMCC 1.3116T, JCM 35326T, ZJ99",JBEHGJ000000000,ZJ151,Pu et al. 2025,3161189.0,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
7,Streptococcus jiangjianxini,Streptococcus jiangjianxini,"ZJ151T, GDMCC 1.3116T, JCM 35326T, ZJ99",JBEHGL000000000,ZJ99,Pu et al. 2025,3161189.0,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
8,Streptococcus handemini,Streptococcus handemini,"ZJ93T, GDMCC 1.3112T, JCM 35325T, ZJ93",JBEHGM000000000,ZJ100,Pu et al. 2025,3161188.0,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
9,Aestuariibius violaceus,Aestuariibius violaceus,"2305UL40-4T, KCCM 43505T, MCCC 1K09164T",JBFONT000000000,2305UL40-4,Lee et al. 2025,3234132.0,https://doi.org/10.1099/ijsem.0.006834,http://dx.doi.org/10.1099/ijsem.0.006834


In [16]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,filtered_url
0,Oceaniferula spumae,Oceaniferula spumae,NT6NT,AP026866,NT6N,Huang et al. 2025,2979115.000000,https://doi.org/10.1099/ijsem.0.006828,http://dx.doi.org/10.1099/ijsem.0.006828
1,Streptococcus tangpeifui,Streptococcus sp. ZJ 1593,"ZJ1593T, GDMCC 1.3115T, JCM 35327T, ZJ373",CP172427,ZJ 1593,Pu et al. 2025,3366265.000000,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
2,Brachymonas wangyanguii,Brachymonas sp. G13,"G13T, GDMCC 1.4691T, JCM 37746T",CP173677,G13,Gao et al. 2025,3130163.000000,https://doi.org/10.1099/ijsem.0.006837,http://dx.doi.org/10.1099/ijsem.0.006837
3,Streptococcus tangpeifui,Streptococcus tangpeifui,"ZJ1593T, GDMCC 1.3115T, JCM 35327T, ZJ373",JAAKDU000000000,ZJ373,Pu et al. 2025,2709400.000000,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
4,Brachymonas wangyanguii,Brachymonas wangyanguii,"G13T, GDMCC 1.4691T, JCM 37746T",JBBEWD000000000,G34,Gao et al. 2025,3130163.000000,https://doi.org/10.1099/ijsem.0.006837,http://dx.doi.org/10.1099/ijsem.0.006837
5,Fodinibius alkaliphilus,Fodinibius alkaliphilus,"N2T, KCTC 102228T, MCCC 1K08942T",JBCFYD000000000,N2,He et al. 2025,3140241.000000,https://doi.org/10.1099/ijsem.0.006840,http://dx.doi.org/10.1099/ijsem.0.006840
6,Streptococcus jiangjianxini,Streptococcus jiangjianxini,"ZJ151T, GDMCC 1.3116T, JCM 35326T, ZJ99",JBEHGJ000000000,ZJ151,Pu et al. 2025,3161189.000000,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
7,Streptococcus jiangjianxini,Streptococcus jiangjianxini,"ZJ151T, GDMCC 1.3116T, JCM 35326T, ZJ99",JBEHGL000000000,ZJ99,Pu et al. 2025,3161189.000000,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
8,Streptococcus handemini,Streptococcus handemini,"ZJ93T, GDMCC 1.3112T, JCM 35325T, ZJ93",JBEHGM000000000,ZJ100,Pu et al. 2025,3161188.000000,https://doi.org/10.1099/ijsem.0.006814,http://dx.doi.org/10.1099/ijsem.0.006814
9,Aestuariibius violaceus,Aestuariibius violaceus,"2305UL40-4T, KCCM 43505T, MCCC 1K09164T",JBFONT000000000,2305UL40-4,Lee et al. 2025,3234132.000000,https://doi.org/10.1099/ijsem.0.006834,http://dx.doi.org/10.1099/ijsem.0.006834


### write output to excel

In [41]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

In [ ]:
combine_df.dtypes